In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import gc, torch
gc.collect()
torch.cuda.empty_cache()

In [3]:
dpath = "/content/drive/MyDrive/rbrtatxscripts/"
rpath = f"{dpath}requirements.txt"
!pip install -q -r {rpath}

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
mdldir = "/content/drive/MyDrive/rbrtftmdlnew"

In [5]:
!pip show transformers

Name: transformers
Version: 4.47.1
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, sentence-transformers


In [29]:
import os, sys, warnings
import torch, random
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import time
from transformers import AutoTokenizer, RobertaForSequenceClassification

def intializeseed(seedvalue=42):
    torch.manual_seed(seedvalue)
    torch.cuda.manual_seed(seedvalue)
    torch.cuda.manual_seed_all(seedvalue)
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = True
    np.random.seed(seedvalue)
    random.seed(seedvalue)
    warnings.filterwarnings('ignore')

def robertatoxicityanalysis(contextstr,mdldir="/content/drive/MyDrive/rbrtftmdlnew",seedvalue=42,threshold='0.020',max_position_embeeding=514,type_vocab_size=1,num_labels=2):
    try :
      max_length = 256
      if contextstr:
          intializeseed(seedvalue)
          device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

          model = RobertaForSequenceClassification.from_pretrained(mdldir,
                                                                  num_labels=num_labels,
                                                                  ).to(device)
          model.eval()
          tokenizer = AutoTokenizer.from_pretrained(mdldir)
          max_length = 256
          inputs = tokenizer(contextstr,return_tensors="pt",padding='max_length', truncation=True)
          #inputs = tokenizer(contextstr, return_tensors="pt")
          validationresponse={}
          threshold = str(threshold)
          with torch.no_grad():
              outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
              logitsobserved = abs(outputs.logits)
              print(f"Logits observed: {logitsobserved}")
              argmaxprediction = logitsobserved.argmax()
              #argmaxprediction = torch.argmax(logitsobserved, dim=1)
              print(f"Observed argmaxprediction: {argmaxprediction}")
              predictedclass = argmaxprediction.item()
              print(f"Predicted class : {predictedclass}")
              cnf_interval = torch.abs(logitsobserved[0][1] - logitsobserved[0][0])

              cnf_intstr = f"{cnf_interval.item():.3f}"
              print(f"Confidence interval: {cnf_interval}")

              validationresponse['evaluationlbl'] = predictedclass
              validationresponse['cnf_intr'] = cnf_intstr
              validationresponse['validationinputstr'] = contextstr

              cnfthresholdanalysis = (cnf_intstr >= threshold)
              validationresponse['thresholdanalysis'] = cnfthresholdanalysis
              if cnfthresholdanalysis:
                if predictedclass == 0:
                    validationresponse['predictionresult'] = 'Toxic'
                    return validationresponse
                else:
                    validationresponse['predictionresult'] = 'Non-Toxic'
                    return validationresponse
              else:
                validationresponse['predictionresult'] = 'Uncertain'
                return validationresponse

    except Exception as e:
        return (f"Encountered error while performing inference: {e}")
        sys.exit(1)

In [30]:
robertatoxicityanalysis(contextstr="You worst fellow")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/rbrtftmdlnew and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Logits observed: tensor([[0.1253, 0.0370]])
Observed argmaxprediction: 0
Predicted class : 0
Confidence interval: 0.08828370273113251


{'evaluationlbl': 0,
 'cnf_intr': '0.088',
 'validationinputstr': 'You worst fellow',
 'thresholdanalysis': True,
 'predictionresult': 'Toxic'}

In [25]:
#with logits
robertatoxicityanalysis(contextstr="You worst fellow")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/rbrtftmdlnew and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Logits observed: tensor([[0.1253, 0.0370]])
Observed argmaxprediction: tensor([0])
Predicted class : 0
Confidence interval: 0.08828370273113251


{'evaluationlbl': 0,
 'cnf_intr': '0.088',
 'validationinputstr': 'You worst fellow',
 'thresholdanalysis': True,
 'predictionresult': 'Toxic'}

In [32]:
robertatoxicityanalysis(contextstr="You worst fellow")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/rbrtftmdlnew and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Logits observed: tensor([[0.1253, 0.0370]])
Observed argmaxprediction: 0
Predicted class : 0
Confidence interval: 0.08828370273113251


{'evaluationlbl': 0,
 'cnf_intr': '0.088',
 'validationinputstr': 'You worst fellow',
 'thresholdanalysis': True,
 'predictionresult': 'Toxic'}

In [31]:
robertatoxicityanalysis(contextstr="You are a good person")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/rbrtftmdlnew and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Logits observed: tensor([[0.0514, 0.0516]])
Observed argmaxprediction: 1
Predicted class : 1
Confidence interval: 0.00018009543418884277


{'evaluationlbl': 1,
 'cnf_intr': '0.000',
 'validationinputstr': 'You are a good person',
 'thresholdanalysis': False,
 'predictionresult': 'Uncertain'}

In [33]:
robertatoxicityanalysis(contextstr="Have a great day")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/rbrtftmdlnew and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Logits observed: tensor([[0.0344, 0.0281]])
Observed argmaxprediction: 0
Predicted class : 0
Confidence interval: 0.006283704191446304


{'evaluationlbl': 0,
 'cnf_intr': '0.006',
 'validationinputstr': 'Have a great day',
 'thresholdanalysis': False,
 'predictionresult': 'Uncertain'}

In [34]:
robertatoxicityanalysis(contextstr="It is great day")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/rbrtftmdlnew and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Logits observed: tensor([[0.0684, 0.0110]])
Observed argmaxprediction: 0
Predicted class : 0
Confidence interval: 0.05732441693544388


{'evaluationlbl': 0,
 'cnf_intr': '0.057',
 'validationinputstr': 'It is great day',
 'thresholdanalysis': True,
 'predictionresult': 'Toxic'}

In [12]:
robertatoxicityanalysis(contextstr="It is a great day")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/rbrtftmdlnew and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Logits observed: tensor([[0.0402, 0.0267]])
Observed argmaxprediction: 0
Predicted class : 0
Confidence interval: 0.01345159113407135


{'evaluationlbl': 0,
 'cnf_intr': '0.013',
 'validationinputstr': 'It is a great day',
 'thresholdanalysis': False,
 'predictionresult': 'Uncertain'}

In [20]:
testlist=["It is a great day", "You are a asshole", "I will kill you", "You should leave",
          "You stupid", "You asshole", "I hate you", "You should die", "You dont deserve to be alive you worst fellow",
          "This is such a horrible move I hate it", "You are piece of shit", "He is definetly a maggot", "He is definetly a maggot..."]

def getresponseasdf(testlist):
    result =[]
    for s in testlist:
      res = robertatoxicityanalysis(s)
      result.append(res)
    rdf = pd.DataFrame(result)
    return rdf

In [21]:
tdf = getresponseasdf(testlist)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/rbrtftmdlnew and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Logits observed: tensor([[0.0402, 0.0267]])
Observed argmaxprediction: tensor([0])
Predicted class : 0
Confidence interval: 0.01345159113407135


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/rbrtftmdlnew and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Logits observed: tensor([[0.1246, 0.0348]])
Observed argmaxprediction: tensor([0])
Predicted class : 0
Confidence interval: 0.0898013785481453


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/rbrtftmdlnew and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Logits observed: tensor([[0.1251, 0.0348]])
Observed argmaxprediction: tensor([0])
Predicted class : 0
Confidence interval: 0.09032373130321503


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/rbrtftmdlnew and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Logits observed: tensor([[0.1244, 0.0359]])
Observed argmaxprediction: tensor([0])
Predicted class : 0
Confidence interval: 0.08846254646778107


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/rbrtftmdlnew and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Logits observed: tensor([[0.1242, 0.0360]])
Observed argmaxprediction: tensor([0])
Predicted class : 0
Confidence interval: 0.08816882967948914


In [22]:
tdf

,evaluationlbl,cnf_intr,validationinputstr,thresholdanalysis,predictionresult
0,0,0.013,It is a great day,False,Uncertain
1,0,0.090,You are a asshole,True,Toxic
2,0,0.087,I will kill you,True,Toxic
3,0,0.292,You should leave,True,Toxic
4,0,0.088,You stupid,True,Toxic
5,0,0.088,You asshole,True,Toxic
6,0,0.089,I hate you,True,Toxic
7,0,0.089,You should die,True,Toxic
8,0,0.088,You dont deserve to be alive you worst fellow,True,Toxic
9,0,0.094,This is such a horrible move I hate it,True,Toxic
